In [2]:
import json
import csv
import numpy as np
import torch
import torch.nn as nn
from torch.utils import data
from tqdm import tqdm_notebook
import torch.optim as optim

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
with open('../../data/processed/masked_full_transcripts.json', 'r') as inp:
    transcripts = json.load(inp)

## Split Data

In [ ]:
np.random.shuffle(transcripts)

In [ ]:
train_split = len(transcripts)*8//10
valid_split = len(transcripts)*1//10
train = transcripts[:train_split]
valid = transcripts[train_split:train_split+valid_split]
test = transcripts[train_split+valid_split:]

train_csv = [('id', 'transcript', 'post_high')]
train_stock = []
for example in train:
    train_csv.append((example['id'], example['transcript'], float(example['post_high_low'][0])))
    historical = map(lambda x: x[1], sorted(example['historical_info'], key=lambda x: x[0]))
    train_stock.append(list(map(lambda x: list(map(float, x)), historical)))
    train_stock[-1] += [example['market_cap'], float(example['post_high_low'][0])]

valid_csv = [('id', 'transcript', 'post_high')]
valid_stock = []
for example in valid:
    valid_csv.append((example['id'], example['transcript'], float(example['post_high_low'][0])))
    historical = map(lambda x: x[1], sorted(example['historical_info'], key=lambda x: x[0]))
    valid_stock.append(list(map(lambda x: list(map(float, x)), historical)))
    valid_stock[-1] += [example['market_cap'], float(example['post_high_low'][0])]

test_csv = [('id', 'transcript', 'post_high')]
test_stock = []
for example in test:
    test_csv.append((example['id'], example['transcript'], float(example['post_high_low'][0])))
    historical = map(lambda x: x[1], sorted(example['historical_info'], key=lambda x: x[0]))
    test_stock.append(list(map(lambda x: list(map(float, x)), historical)))
    test_stock[-1] += [example['market_cap'], float(example['post_high_low'][0])]

In [ ]:
with open('../../data/processed/splits/train/transcripts.csv', 'w') as out:
    writer = csv.writer(out)
    header = None
    for row in train_csv:
        if header is None:
            writer.writerow(row)
            header = row
        else:
            [id, transcript, label] = row
            writer.writerow([id, ' '.join(transcript), label])

with open('../../data/processed/splits/valid/transcripts.csv', 'w') as out:
    writer = csv.writer(out)
    header = None
    for row in valid_csv:
        if header is None:
            writer.writerow(row)
            header = row
        else:
            [id, transcript, label] = row
            writer.writerow([id, ' '.join(transcript), label])

with open('../../data/processed/splits/test/transcripts.csv', 'w') as out:
    writer = csv.writer(out)
    header = None
    for row in test_csv:
        if header is None:
            writer.writerow(row)
            header = row
        else:
            [id, transcript, label] = row
            writer.writerow([id, ' '.join(transcript), label])

In [ ]:
with open('../../data/processed/splits/train/stock_data.json', 'w') as out:
    json.dump(train_stock, out)
with open('../../data/processed/splits/valid/stock_data.json', 'w') as out:
    json.dump(valid_stock, out)
with open('../../data/processed/splits/test/stock_data.json', 'w') as out:
    json.dump(test_stock, out)

In [ ]:
with open('../../data/processed/splits/valid/stock_data.csv', 'w') as out:
    writer = csv.writer(out)
    for row in valid_stock:
        writer.writerow(row)

## Load Data

In [5]:
with open('../../data/processed/splits/train/stock_data.json', 'r') as inp:
    train_stock = json.load(inp)
with open('../../data/processed/splits/valid/stock_data.json', 'r') as inp:
    valid_stock = json.load(inp)
with open('../../data/processed/splits/test/stock_data.json', 'r') as inp:
    test_stock = json.load(inp)

In [6]:
train_transcripts = []
with open('../../data/processed/splits/train/transcripts.csv', 'r') as inp:
    reader = csv.reader(inp)
    headers = None
    for row in reader:
        if headers is None:
            headers = row
        else:
            row[1] = row[1].split(' ')
            train_transcripts.append(row)

## Create Dataset

In [6]:
class StockDataset(data.Dataset):
    def __init__(self, examples):
        examples = np.array(examples)
        self.labels = examples[:,-1]
        self.market_cap = examples[:,-2]
        self.examples = np.array(examples[:,:-2].tolist())

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.examples)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        X = torch.tensor(self.examples[index])
        auxiliary = self.market_cap[index]
        y = self.labels[index]

        return X, auxiliary, y

In [7]:
valid_dataset = StockDataset(valid_stock)

In [8]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}
valid_generator = data.DataLoader(valid_dataset, **params)

## Train Model

In [9]:
class BaselineStockPredictor(nn.Module):
    """
    Model that will read in plain stock ticker values over time and decide whether to buy, sell, or hold at the current price.
    """
    def __init__(self, num_series_features=1, num_auxiliary_features=1, hidden_size=128, output_size=1):
        """
        Attributes:
            num_series_features: the size of the feature set for an individual
                                 stock price example (e.g. if we include high,
                                 low, average, num_series_features will equal 3
            num_auxiliary_features: the number of auxiliary (not dependent on time)
                                    features we are adding (e.g. if we include the 1yr
                                    high and the market capitalization, num_auxiliary_features
                                    would equal 2
            output_size: the size of the outputted vector. For evaluation, we would use a
                         size of 1 (stock price) or 3 (buy, sell, hold classification).
                         For use in the looking glass model, we want an encoding so we might
                         use a size of 128 to feed into the model.
        """
        super().__init__()
        self.recurrent = nn.LSTM(
            input_size=num_series_features,
            hidden_size=hidden_size,
            num_layers=1,
            bidirectional=False,
            batch_first=True,
            dropout=0.5
        )
        # concatenate LSTM output with auxiliary features
        # output predicted price
        self.linear = nn.Linear(hidden_size+num_auxiliary_features, output_size)
        self.init_weights()

    def init_weights(self):
        """
        Initializes the weights of the model
        """
        for layer in [self.linear]:
            nn.init.xavier_uniform_(layer.weight)
            nn.init.constant_(layer.bias, 0.0)

    def forward(self, X_series, X_auxiliary):
        """
        Moves the model through each layer
        Parameters:
            X_series: an [N, num_series_examples, num_series_features] size vector
                      where N is the batch size, num_series_examples is how many stock prices
                      we are providing per example (e.g. weekly for the last 3 months), and
                      num_series_features is the same as described in __init__
            X_auxiliary: an [N, num_auxiliary_features] vector
        """
        recurrent_output,_ = self.recurrent(X_series)
        recurrent_output = torch.mean(recurrent_output, 1)
        # We might need this
        # recurrent_output = torch.squeeze(1) 
        aux_combined = torch.cat([recurrent_output, X_auxiliary], dim=1)
        output = self.linear(aux_combined)

        return output

In [10]:
def get_stock_iterator(input_data, batch_size, train=True, shuffle=True):
    dataset = StockDataset(input_data)
    iterator = data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=5)
    return iterator
    
def train_model(train, valid, num_epochs=200, learning_rate=0.003, existing_model=None):
    batch_size = 128
    train_iterator = get_stock_iterator(train, batch_size)
    valid_iterator = get_stock_iterator(valid, batch_size)
    
    model = None
    
    if existing_model is None:
        model = BaselineStockPredictor(num_series_features=2, hidden_size=64)
    else:
        model = existing_model
    model = model.float().cuda()
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), learning_rate)
    
    losses = []
    valid_scores = []
    
    min_mse = float('inf')
    delay = 0
    MAX_INC = 20
    
    for epoch in tqdm_notebook(range(num_epochs)):
        model.train()
        iter_losses = []
        for batch_series, batch_aux, batch_labels in train_iterator:
            batch_aux = torch.reshape(batch_aux, (-1,1))
            optimizer.zero_grad()
            outputs = model(batch_series.float().cuda(), batch_aux.float().cuda())
            batch_labels = torch.reshape(batch_labels, (-1,1))
            loss = criterion(outputs, batch_labels.float().cuda())
            loss.backward()
            optimizer.step()
            iter_losses.append(loss.item())
        iter_losses = np.array(iter_losses)
        losses.append(np.mean(iter_losses))
        
        valid_mse = []
        model.eval()

        for valid_batch_series, valid_batch_aux, valid_batch_labels in valid_iterator:
            valid_batch_aux = torch.reshape(valid_batch_aux, (-1,1))
            outputs = model(valid_batch_series.float().cuda(), valid_batch_aux.float().cuda())
            valid_batch_labels = torch.reshape(valid_batch_labels, (-1,1))
            loss = criterion(outputs, valid_batch_labels.float().cuda())
            valid_mse.append(loss.item())
        valid_mse = np.mean(valid_mse)
        print(f'Completed epoch {epoch}. Valid MSE: {valid_mse}')


        if valid_mse < min_mse:
            min_mse = valid_mse
            delay = 0
            torch.save(model, 'model.ckpt')
        else:
            delay += 1
    
    return model, losses

In [ ]:
model, losses = train_model(train_stock, valid_stock, num_epochs=10000)

/home/seanstapleton/ml/lib/python3.7/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Completed epoch 0. Valid MSE: 32013.314453125


/home/seanstapleton/ml/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BaselineStockPredictor. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/seanstapleton/ml/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/seanstapleton/ml/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Completed epoch 1. Valid MSE: 31528.376953125
Completed epoch 2. Valid MSE: 31025.55078125
Completed epoch 3. Valid MSE: 30445.822265625
Completed epoch 4. Valid MSE: 29766.986328125
Completed epoch 5. Valid MSE: 29127.77734375
Completed epoch 6. Valid MSE: 28701.177734375
Completed epoch 7. Valid MSE: 28284.650390625
Completed epoch 8. Valid MSE: 27831.666015625
Completed epoch 9. Valid MSE: 27359.376953125
Completed epoch 10. Valid MSE: 26891.03515625
Completed epoch 11. Valid MSE: 26474.033203125
Completed epoch 12. Valid MSE: 26082.15625
Completed epoch 13. Valid MSE: 25684.43359375
Completed epoch 14. Valid MSE: 25303.71875
Completed epoch 15. Valid MSE: 24948.28515625
Completed epoch 16. Valid MSE: 24620.185546875
Completed epoch 17. Valid MSE: 24306.05859375
Completed epoch 18. Valid MSE: 23996.466796875
Completed epoch 19. Valid MSE: 23703.619140625
Completed epoch 20. Valid MSE: 23427.94921875
Completed epoch 21. Valid MSE: 23163.13671875
Completed epoch 22. Valid MSE: 22913.3

Completed epoch 176. Valid MSE: 13800.33203125
Completed epoch 177. Valid MSE: 13794.4501953125
Completed epoch 178. Valid MSE: 13777.908203125
Completed epoch 179. Valid MSE: 13702.8359375
Completed epoch 180. Valid MSE: 13680.2197265625
Completed epoch 181. Valid MSE: 13674.9443359375
Completed epoch 182. Valid MSE: 13624.259765625
Completed epoch 183. Valid MSE: 13601.234375
Completed epoch 184. Valid MSE: 13547.6943359375
Completed epoch 185. Valid MSE: 13545.5517578125
Completed epoch 186. Valid MSE: 13454.64453125
Completed epoch 187. Valid MSE: 13449.7109375
Completed epoch 188. Valid MSE: 13516.3388671875
Completed epoch 189. Valid MSE: 13554.9287109375
Completed epoch 190. Valid MSE: 13371.466796875
Completed epoch 191. Valid MSE: 13316.8642578125
Completed epoch 192. Valid MSE: 13288.8193359375
Completed epoch 193. Valid MSE: 13336.9775390625
Completed epoch 194. Valid MSE: 13289.2138671875
Completed epoch 195. Valid MSE: 13183.990234375
Completed epoch 196. Valid MSE: 13140.

...

Completed epoch 4098. Valid MSE: 688.8971557617188
Completed epoch 4099. Valid MSE: 714.9573974609375
Completed epoch 4100. Valid MSE: 697.1425170898438
Completed epoch 4101. Valid MSE: 698.6249389648438
Completed epoch 4102. Valid MSE: 698.9046020507812
Completed epoch 4103. Valid MSE: 688.8692626953125
Completed epoch 4104. Valid MSE: 687.0731201171875
Completed epoch 4105. Valid MSE: 691.5723876953125
Completed epoch 4106. Valid MSE: 671.7390747070312
Completed epoch 4107. Valid MSE: 674.1051025390625
Completed epoch 4108. Valid MSE: 689.2382202148438
Completed epoch 4109. Valid MSE: 681.8084106445312
Completed epoch 4110. Valid MSE: 688.8060913085938
Completed epoch 4111. Valid MSE: 686.3260498046875
Completed epoch 4112. Valid MSE: 665.6329956054688
Completed epoch 4113. Valid MSE: 669.8432006835938
Completed epoch 4114. Valid MSE: 659.8409423828125
Completed epoch 4115. Valid MSE: 617.3056640625
Completed epoch 4116. Valid MSE: 628.5809326171875
Completed epoch 4117. Valid MSE: 7

Completed epoch 4260. Valid MSE: 1451.4207763671875
Completed epoch 4261. Valid MSE: 1451.77001953125
Completed epoch 4262. Valid MSE: 1412.6385498046875
Completed epoch 4263. Valid MSE: 1400.6405029296875
Completed epoch 4264. Valid MSE: 1405.8670654296875
Completed epoch 4265. Valid MSE: 1383.732666015625
Completed epoch 4266. Valid MSE: 1372.4912109375
Completed epoch 4267. Valid MSE: 1421.7344970703125
Completed epoch 4268. Valid MSE: 1395.0457763671875
Completed epoch 4269. Valid MSE: 1401.2415771484375
Completed epoch 4270. Valid MSE: 1391.9718017578125
Completed epoch 4271. Valid MSE: 1369.8619384765625
Completed epoch 4272. Valid MSE: 1409.048828125
Completed epoch 4273. Valid MSE: 1393.16943359375
Completed epoch 4274. Valid MSE: 1432.943603515625
Completed epoch 4275. Valid MSE: 1472.8712158203125
Completed epoch 4276. Valid MSE: 1461.4736328125
Completed epoch 4277. Valid MSE: 1455.13623046875
Completed epoch 4278. Valid MSE: 1476.86962890625
Completed epoch 4279. Valid MSE:

Completed epoch 4421. Valid MSE: 1968.14794921875
Completed epoch 4422. Valid MSE: 2243.072509765625
Completed epoch 4423. Valid MSE: 2170.485595703125
Completed epoch 4424. Valid MSE: 2023.8292236328125
Completed epoch 4425. Valid MSE: 1905.935302734375
Completed epoch 4426. Valid MSE: 1893.4560546875
Completed epoch 4427. Valid MSE: 1857.0538330078125
Completed epoch 4428. Valid MSE: 1757.0716552734375
Completed epoch 4429. Valid MSE: 1710.4493408203125
Completed epoch 4430. Valid MSE: 1554.169921875
Completed epoch 4431. Valid MSE: 1540.675048828125
Completed epoch 4432. Valid MSE: 1542.17626953125
Completed epoch 4433. Valid MSE: 1501.481201171875
Completed epoch 4434. Valid MSE: 1583.2388916015625
Completed epoch 4435. Valid MSE: 1918.5587158203125
Completed epoch 4436. Valid MSE: 1861.27783203125
Completed epoch 4437. Valid MSE: 1873.9630126953125
Completed epoch 4438. Valid MSE: 1959.7652587890625
Completed epoch 4439. Valid MSE: 1979.35205078125
Completed epoch 4440. Valid MSE:

Completed epoch 4584. Valid MSE: 2003.302978515625
Completed epoch 4585. Valid MSE: 2014.3397216796875
Completed epoch 4586. Valid MSE: 2038.10107421875
Completed epoch 4587. Valid MSE: 2052.9169921875
Completed epoch 4588. Valid MSE: 2062.168701171875
Completed epoch 4589. Valid MSE: 2014.135009765625
Completed epoch 4590. Valid MSE: 1987.712890625
Completed epoch 4591. Valid MSE: 1972.9776611328125
Completed epoch 4592. Valid MSE: 1982.655029296875
Completed epoch 4593. Valid MSE: 1973.0093994140625
Completed epoch 4594. Valid MSE: 1915.175537109375
Completed epoch 4595. Valid MSE: 1804.545166015625
Completed epoch 4596. Valid MSE: 1666.7723388671875
Completed epoch 4597. Valid MSE: 1570.4166259765625
Completed epoch 4598. Valid MSE: 1496.2276611328125
Completed epoch 4599. Valid MSE: 1397.2535400390625
Completed epoch 4600. Valid MSE: 1333.20849609375
Completed epoch 4601. Valid MSE: 1315.3917236328125
Completed epoch 4602. Valid MSE: 1254.9388427734375
Completed epoch 4603. Valid M

Completed epoch 4746. Valid MSE: 1626.6470947265625
Completed epoch 4747. Valid MSE: 1628.2667236328125
Completed epoch 4748. Valid MSE: 1631.7940673828125
Completed epoch 4749. Valid MSE: 1634.8450927734375
Completed epoch 4750. Valid MSE: 1644.40380859375
Completed epoch 4751. Valid MSE: 1654.5677490234375
Completed epoch 4752. Valid MSE: 1664.2200927734375
Completed epoch 4753. Valid MSE: 1660.7269287109375
Completed epoch 4754. Valid MSE: 1654.900390625
Completed epoch 4755. Valid MSE: 1656.402587890625
Completed epoch 4756. Valid MSE: 1667.998779296875
Completed epoch 4757. Valid MSE: 1672.341796875
Completed epoch 4758. Valid MSE: 1654.9373779296875
Completed epoch 4759. Valid MSE: 1639.38720703125
Completed epoch 4760. Valid MSE: 1649.78662109375
Completed epoch 4761. Valid MSE: 1633.7130126953125
Completed epoch 4762. Valid MSE: 1606.9141845703125
Completed epoch 4763. Valid MSE: 1600.58056640625
Completed epoch 4764. Valid MSE: 1579.3309326171875
Completed epoch 4765. Valid MS

In [36]:
def eval_model(model, test):
    batch_size = 128
    test_iterator = get_stock_iterator(test, batch_size, shuffle=False)
        
    criterion = nn.MSELoss()
    
    model.eval()
    losses = []
    for batch_series, batch_aux, batch_labels in test_iterator:
        batch_aux = torch.reshape(batch_aux, (-1,1))
        outputs = model(batch_series.float().cuda(), batch_aux.float().cuda())
        batch_labels = torch.reshape(batch_labels, (-1,1))
        loss = criterion(outputs, batch_labels.float().cuda())
        losses.append(loss.item())
    return np.mean(losses)

In [41]:
eval_model(updated_model, test_stock)

2511.2421875

In [1]:
print('hi')

hi
